# Market Discovery - Scraping & LLM Experiments

Run scraping experiments interactively with Jupyter!

In [2]:
pip install -U langchain &> /dev/null &

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U langchain-openai &> /dev/null

Note: you may need to restart the kernel to use updated packages.


In [7]:
import sys
sys.path.insert(0, '/app')
import asyncio
import csv
import sys
from datetime import datetime
from pathlib import Path
from typing import List
import pandas as pd 

from loguru import logger
from libs.common.models import Listing


AVAILABLE_CONNECTORS = {
    "ebay": "ingestion.connectors.ebay",
    "leboncoin": "ingestion.connectors.leboncoin",
    "vinted": "ingestion.connectors.vinted",
    "backmarket": "ingestion.connectors.backmarket_connector",
    "cdiscount": "ingestion.connectors.cdiscount_connector",
    "fnac": "ingestion.connectors.fnac_connector",
    "rakuten": "ingestion.connectors.rakuten_connector",
}

CONNECTOR_CLASSES = {
    "ebay": None,  # Uses functions, not a class
    "leboncoin": "LeBonCoinConnector",
    "vinted": "VintedConnector",
    "backmarket": "BackmarketConnector",
    "cdiscount": "CdiscountConnector",
    "fnac": "FnacConnector",
    "rakuten": "RakutenConnector",
}


def get_connector(connector_name: str):
    """Dynamically import and instantiate a connector"""
    if connector_name not in AVAILABLE_CONNECTORS:
        raise ValueError(f"Unknown connector: {connector_name}. Available: {', '.join(AVAILABLE_CONNECTORS.keys())}")
    
    module_path = AVAILABLE_CONNECTORS[connector_name]
    class_name = CONNECTOR_CLASSES[connector_name]
    
    # Import the module
    import importlib
    module = importlib.import_module(module_path)
    
    # Get the connector class or function
    if class_name:
        connector_class = getattr(module, class_name)
        return connector_class()
    else:
        # For eBay, return the module (uses functions)
        return module


async def search_with_connector(connector, connector_name: str, keyword: str, limit: int = 20) -> List[Listing]:
    """Search using the appropriate connector method"""
    if connector_name == "ebay":
        from ingestion.connectors.ebay import fetch_ebay_listings
        return await fetch_ebay_listings(keyword, limit=limit)
    else:
        # Other connectors use class-based approach
        return await connector.search_items(keyword, limit=limit)

In [10]:
keyword = "Sony PS5"
limit = 5
provider = "fnac"


In [11]:
connector = get_connector(provider)
listings = await search_with_connector(connector, provider, keyword, limit=limit)
listings_df = pd.DataFrame([dict(l) for l in listings])
listings_df

2025-11-04 16:02:12.294 | INFO     | ingestion.connectors.fnac_connector:search_items:128 - Searching Fnac for: Sony PS5
2025-11-04 16:02:38.412 | INFO     | ingestion.connectors.fnac_connector:search_items:161 - Fnac search returned 5 items for 'Sony PS5'


,source,listing_id,title,description,price,currency,condition_raw,condition_norm,location,seller_rating,shipping_cost,observed_at,is_sold,url,brand,size,color
0,fnac,22167261,Pack Console Sony PS®5 Slim Edition Standard B...,None,549.99,EUR,None,None,None,None,None,2025-11-04 15:02:38.412597+00:00,False,https://www.fnac.com/Pack-Console-Sony-PS-5-Sl...,None,None,None
1,fnac,22167263,Pack Console Sony PS®5 Slim Edition Digital Bl...,None,568.99,EUR,None,None,None,None,None,2025-11-04 15:02:38.412633+00:00,False,https://www.fnac.com/Pack-Console-Sony-PS-5-Sl...,None,None,None
2,fnac,18919798,Console Sony PS5 Slim Edition Standard Blanc e...,None,529.99,EUR,None,None,None,None,None,2025-11-04 15:02:38.412641+00:00,False,https://www.fnac.com/Console-Sony-PS5-Slim-Edi...,None,None,None
3,fnac,21985466,Console Sony PS5 Slim Edition Digital Blanc,None,449.99,EUR,None,None,None,None,None,2025-11-04 15:02:38.412648+00:00,False,https://www.fnac.com/Console-Sony-PS5-Slim-Edi...,None,None,None
4,fnac,21925913,Pack Console PlayStation 5 Sony Edition Standa...,None,494.99,EUR,None,None,None,None,None,2025-11-04 15:02:38.412654+00:00,False,https://www.fnac.com/Pack-Console-PlayStation-...,None,None,None


In [12]:
listings[0].url

'https://www.fnac.com/Pack-Console-Sony-PS-5-Slim-Edition-Standard-Blanc-EA-SPORTS-FC-26/a22167261/w-4'

{'source': 'cdiscount',
 'listing_id': 'f-1035001-hbps5stdfc26dsw.html#mpos=1|cd',
 'title': 'Pack PS5 Standard : Console PlayStation 5 (modèle Slim) + FC26 (code dans la boite) + 2ème manette DualSense Blanche',
 'description': None,
 'price': 609.0,
 'currency': 'EUR',
 'condition_raw': None,
 'condition_norm': None,
 'location': None,
 'seller_rating': None,
 'shipping_cost': None,
 'observed_at': datetime.datetime(2025, 11, 4, 13, 39, 9, 544447, tzinfo=datetime.timezone.utc),
 'is_sold': False,
 'url': 'https://www.cdiscount.com/jeux-pc-video-console/ps5/pack-ps5-standard-console-playstation-5-modele/f-1035001-hbps5stdfc26dsw.html#mpos=1|cd',
 'brand': None,
 'size': None,
 'color': None}